In [21]:
import os
import json
import tomli
import numpy as np
import pandas as pd
from datavis import METHOD_MAPPER, DATASET_MAPPER

In [22]:
EVAL_PATH = './eval/'
RESULTS = {}

In [23]:
def load_config(path) -> dict:
    with open(path, 'rb') as f:
        return tomli.load(f)
    
def load_json(js_path) -> dict:
    with open(js_path, 'r') as f:
        return json.load(f)

In [24]:
def print_config():
    config = load_config('assess.toml')
    considered = config['methods']['considered']
    RESULTS['methods'] = considered
    print(considered)

print_config()

['codi', 'fairsmote', 'fairtabddpm', 'fairtabgan', 'goggle', 'great', 'smote', 'stasy', 'tabddpm', 'tabsyn']


In [25]:
def read_score(dataset, stats=False):
    score_path = os.path.join(EVAL_PATH, 'quality', dataset, 'score.json')
    dcr_path = os.path.join(EVAL_PATH, 'quality', dataset, 'dcr.json')

    score = load_json(score_path)
    dcr = load_json(dcr_path)
    for data in score:
        score[data]['dcr'] = dcr[data]
    
    score_stats = {}
    if stats:
        for data in score:
            score_stats[data] = {}
            for metric in score[data]:
                if metric not in score_stats:
                    mu, sigma = np.mean(score[data][metric]), np.std(score[data][metric])
                    mu, sigma = round(mu, 3), round(sigma, 3)
                    score_stats[data][metric] = (mu, sigma)
        return score_stats
    
    return score

In [26]:
def convert_tuple(t):
    ans = (1 - t[0], t[1])
    ans = (round(ans[0], 3), round(ans[1], 3))
    return ans

def convert_to_latex(t):    
    ans = f"${t[0]:.3f}_{{\pm {t[1]:.3f}}}$"
    # eliminate the leading zero
    ans = ans.replace('0.', '.')
    return ans

In [27]:
def get_tex_score_df_dataset(dataset, idx_order):
    score = read_score(dataset, stats=True)
    # metrics = ['shape', 'trend', 'dcr']
    metrics = ['shape', 'trend']
    score_df = pd.DataFrame(score).T[metrics]

    # rename the columns
    ds_name = DATASET_MAPPER[dataset]
    score_df = score_df.rename(columns={
        'shape': f'{ds_name} - Density',
        'trend': f'{ds_name} - Correlation',
    })

    # rearrange the index order
    score_df = score_df.reindex(idx_order)

    # rename index according to the method mapper
    score_df = score_df.rename(index=METHOD_MAPPER)

    # convert the tuple to latex
    score_df = score_df.map(convert_tuple).map(convert_to_latex)
    return score_df

In [28]:
idx_order = [
    'codi', 'goggle', 'great', 'smote', 'stasy',
    'tabddpm', 'tabsyn', 'fairsmote', 'fairtabgan', 'fairtabddpm',
]

In [29]:
adult_df = get_tex_score_df_dataset('adult', idx_order)
compas_df = get_tex_score_df_dataset('compass', idx_order)
bank_df = get_tex_score_df_dataset('bank', idx_order)

In [30]:
all_df = pd.concat([adult_df, compas_df, bank_df], axis=1)
density_cols = [col for col in all_df.columns if 'Density' in col]
correlation_cols = [col for col in all_df.columns if 'Correlation' in col]
all_df['Ave. - Density'] = all_df[density_cols].apply(lambda x: np.mean([float(i.split('_')[0][1:]) for i in x]), axis=1)
all_df['Ave. - Correlation'] = all_df[correlation_cols].apply(lambda x: np.mean([float(i.split('_')[0][1:]) for i in x]), axis=1)

# to percentage
all_df['Ave. - Density'] = all_df['Ave. - Density'].map(lambda x: x * 100)
all_df['Ave. - Correlation'] = all_df['Ave. - Correlation'].map(lambda x: x * 100)

all_df['Ave. - Density'] = all_df['Ave. - Density'].map(lambda x: f"${x:.1f}\%$")
all_df['Ave. - Correlation'] = all_df['Ave. - Correlation'].map(lambda x: f"${x:.1f}\%$")

# reorder the columns
all_df = all_df[
    [
        'Adult - Density', 'Bank - Density', 'COMPAS - Density', 'Ave. - Density',
        'Adult - Correlation', 'Bank - Correlation', 'COMPAS - Correlation', 'Ave. - Correlation',
    ]
]
all_df.head(2)

,Adult - Density,Bank - Density,COMPAS - Density,Ave. - Density,Adult - Correlation,Bank - Correlation,COMPAS - Correlation,Ave. - Correlation
CoDi,$.145_{\pm .000}$,$.154_{\pm .000}$,$.205_{\pm .001}$,$16.8\%$,$.495_{\pm .000}$,$.344_{\pm .001}$,$.550_{\pm .001}$,$46.3\%$
Goggle,$.158_{\pm .001}$,$.131_{\pm .000}$,$.164_{\pm .001}$,$15.1\%$,$.447_{\pm .016}$,$.232_{\pm .000}$,$.332_{\pm .022}$,$33.7\%$


In [31]:
methods = all_df.index
max_str_len = max([len(m) for m in methods])

for i in range(all_df.shape[0]):
    method = all_df.index[i]
    n_slash = max_str_len - len(method)
    print(
        method + ' ' * n_slash, '&',
        all_df.iloc[i].values[0], '&',
        all_df.iloc[i].values[1], '&',
        all_df.iloc[i].values[2], '&',
        all_df.iloc[i].values[3], '&',
        all_df.iloc[i].values[4], '&',
        all_df.iloc[i].values[5], '&',
        all_df.iloc[i].values[6], '&',
        all_df.iloc[i].values[7], '\\\\',
    )

CoDi     & $.145_{\pm .000}$ & $.154_{\pm .000}$ & $.205_{\pm .001}$ & $16.8\%$ & $.495_{\pm .000}$ & $.344_{\pm .001}$ & $.550_{\pm .001}$ & $46.3\%$ \\
Goggle   & $.158_{\pm .001}$ & $.131_{\pm .000}$ & $.164_{\pm .001}$ & $15.1\%$ & $.447_{\pm .016}$ & $.232_{\pm .000}$ & $.332_{\pm .022}$ & $33.7\%$ \\
GReaT    & $.077_{\pm .000}$ & $.102_{\pm .001}$ & $.093_{\pm .001}$ & $9.1\%$ & $.208_{\pm .015}$ & $.213_{\pm .010}$ & $.165_{\pm .016}$ & $19.5\%$ \\
SMOTE    & $.025_{\pm .000}$ & $.020_{\pm .000}$ & $.021_{\pm .001}$ & $2.2\%$ & $.054_{\pm .004}$ & $.042_{\pm .005}$ & $.047_{\pm .005}$ & $4.8\%$ \\
STaSy    & $.102_{\pm .000}$ & $.182_{\pm .000}$ & $.108_{\pm .001}$ & $13.1\%$ & $.163_{\pm .000}$ & $.221_{\pm .001}$ & $.138_{\pm .001}$ & $17.4\%$ \\
TabDDPM  & $.037_{\pm .001}$ & $.028_{\pm .001}$ & $.057_{\pm .001}$ & $4.1\%$ & $.055_{\pm .001}$ & $.052_{\pm .001}$ & $.090_{\pm .001}$ & $6.6\%$ \\
TabSyn   & $.010_{\pm .000}$ & $.009_{\pm .000}$ & $.027_{\pm .001}$ & $1.5\%$ & 

In [15]:
def read_auc(dataset, method, mode='original', stats=False):
    js_path = os.path.join(EVAL_PATH, 'learning', dataset, f'best_{mode}_{method}.json')
    scores = {}
    ans = load_json(js_path)['CatBoost']
    for key in ans:
        scores[key] = ans[key]['Test']
    scores = scores['AUC']
    
    if stats:
        mu, sigma = np.mean(scores), np.std(scores)
        mu, sigma = round(mu, 3), round(sigma, 3)
        return (mu, sigma)
    
    return scores

In [16]:
def read_all_auc(mode='original'):
    datasets = ['adult', 'bank', 'compass']
    methods = RESULTS['methods'] + ['real']
    auc_scores = {}
    for dataset in datasets:
        auc_scores[dataset] = {}
        for method in methods:
            auc = read_auc(dataset, method, mode=mode, stats=True)
            auc_scores[dataset][method] = auc
    return auc_scores

In [17]:
def to_latex_percentage(num):
    # input is a float
    return f"${num:.1f}\%$"

In [18]:
auc_df = pd.DataFrame(read_all_auc())
# average the first elment of the tuple in the dataframe
auc_df['average'] = auc_df.apply(lambda x: np.mean([i[0] for i in x.values]), axis=1)

# convert to latex for adult, bank, and compass
for dataset in ['adult', 'bank', 'compass']:
    for method in RESULTS['methods']:
        auc_df[dataset][method] = convert_to_latex(auc_df[dataset][method])
    auc_df[dataset]['real'] = convert_to_latex(auc_df[dataset]['real'])

# transform the average to differnce with real as the baseline
# auc_df['average'] = (auc_df['average'] - auc_df['average']['real']) / auc_df['average']['real']

# convert average to percentage and then to latex
auc_df['average'] = auc_df['average'] * 100 

# convert to latex
auc_df['average'] = auc_df['average'].map(to_latex_percentage)

# rename the columns
auc_df = auc_df.rename(columns={
    'average': 'Average',
    'adult': 'Adult',
    'bank': 'Bank',
    'compass': 'COMPAS',
})

# reorder the index
auc_df = auc_df.reindex(['real'] + idx_order)

# rename the index
auc_df = auc_df.rename(index=METHOD_MAPPER)

methods = auc_df.index
max_str_len = max([len(m) for m in methods])

for i in range(auc_df.shape[0]):
    method = auc_df.index[i]
    n_slash = max_str_len - len(method)
    print(
        method + ' ' * n_slash, '&',
        auc_df.iloc[i].values[0], '&',
        auc_df.iloc[i].values[1], '&',
        auc_df.iloc[i].values[2], '&',
        auc_df.iloc[i].values[3], '\\\\',
    )

Real     & $.928_{\pm .000}$ & $.936_{\pm .000}$ & $.810_{\pm .001}$ & $89.1\%$ \\
CoDi     & $.858_{\pm .001}$ & $.826_{\pm .014}$ & $.678_{\pm .002}$ & $78.7\%$ \\
Goggle   & $.740_{\pm .002}$ & $.737_{\pm .006}$ & $.650_{\pm .009}$ & $70.9\%$ \\
GReaT    & $.901_{\pm .002}$ & $.688_{\pm .024}$ & $.717_{\pm .008}$ & $76.9\%$ \\
SMOTE    & $.914_{\pm .000}$ & $.928_{\pm .001}$ & $.778_{\pm .002}$ & $87.3\%$ \\
STaSy    & $.885_{\pm .003}$ & $.895_{\pm .003}$ & $.728_{\pm .013}$ & $83.6\%$ \\
TabDDPM  & $.907_{\pm .001}$ & $.917_{\pm .002}$ & $.745_{\pm .001}$ & $85.6\%$ \\
TabSyn   & $.911_{\pm .000}$ & $.919_{\pm .000}$ & $.749_{\pm .001}$ & $86.0\%$ \\
FairCB   & $.915_{\pm .001}$ & $.907_{\pm .002}$ & $.771_{\pm .001}$ & $86.4\%$ \\
FairTGAN & $.881_{\pm .000}$ & $.863_{\pm .006}$ & $.705_{\pm .002}$ & $81.6\%$ \\
Ours     & $.893_{\pm .002}$ & $.914_{\pm .002}$ & $.734_{\pm .001}$ & $84.7\%$ \\


In [19]:
def read_fair(dataset, method, mode='original', metric='DPR', stats=False):
    js_path = os.path.join(EVAL_PATH, 'learning', dataset, f'best_{mode}_{method}.json')
    scores = {}
    ans = load_json(js_path)['CatBoost']
    for key in ans:
        scores[key] = ans[key]['Test']
    
    if dataset == 'adult' or dataset == 'compass':
        attri = 'sex'
    elif dataset == 'bank':
        attri = 'age-group'
        
    scores = scores[metric][attri]
    
    if stats:
        mu, sigma = np.mean(scores), np.std(scores)
        mu, sigma = round(mu, 3), round(sigma, 3)
        return (mu, sigma)
    
    return scores

In [20]:
dpr_dict = {}
for dataset in ['adult', 'bank', 'compass']:
    dpr_dict[dataset] = {}
    for method in ['real'] + RESULTS['methods']:
        dpr = read_fair(dataset, method, metric='DPR', stats=True)
        dpr_dict[dataset][method] = dpr

dpr_df = pd.DataFrame(dpr_dict)
dpr_df['average'] = dpr_df.apply(lambda x: np.mean([i[0] for i in x.values]), axis=1)

# convert to latex for adult, bank, and compass
for dataset in ['adult', 'bank', 'compass']:
    for method in RESULTS['methods']:
        dpr_df[dataset][method] = convert_to_latex(dpr_df[dataset][method])
    dpr_df[dataset]['real'] = convert_to_latex(dpr_df[dataset]['real'])

dpr_df['average'] = dpr_df['average'] * 100 
dpr_df['average'] = dpr_df['average'].map(to_latex_percentage)

eor_dict = {}
for dataset in ['adult', 'bank', 'compass']:
    eor_dict[dataset] = {}
    for method in ['real'] + RESULTS['methods']:
        eor = read_fair(dataset, method, metric='EOR', stats=True)
        eor_dict[dataset][method] = eor
        
eor_df = pd.DataFrame(eor_dict)
eor_df['average'] = eor_df.apply(lambda x: np.mean([i[0] for i in x.values]), axis=1)

# convert to latex for adult, bank, and compass
for dataset in ['adult', 'bank', 'compass']:
    for method in RESULTS['methods']:
        eor_df[dataset][method] = convert_to_latex(eor_df[dataset][method])
    eor_df[dataset]['real'] = convert_to_latex(eor_df[dataset]['real'])

eor_df['average'] = eor_df['average'] * 100
eor_df['average'] = eor_df['average'].map(to_latex_percentage)

# reorder the index
dpr_df = dpr_df.reindex(['real'] + idx_order)
eor_df = eor_df.reindex(['real'] + idx_order)

# rename the index
dpr_df = dpr_df.rename(index=METHOD_MAPPER)
eor_df = eor_df.rename(index=METHOD_MAPPER)

fair_df = pd.concat([dpr_df, eor_df], axis=1)

for i in range(fair_df.shape[0]):
    method = fair_df.index[i]
    n_slash = max_str_len - len(method)
    print(
        method + ' ' * n_slash, '&',
        fair_df.iloc[i].values[0], '&',
        fair_df.iloc[i].values[1], '&',
        fair_df.iloc[i].values[2], '&',
        fair_df.iloc[i].values[3], '&',
        fair_df.iloc[i].values[4], '&',
        fair_df.iloc[i].values[5], '&',
        fair_df.iloc[i].values[6], '&',
        fair_df.iloc[i].values[7], '\\\\',
    )

Real     & $.309_{\pm .001}$ & $.402_{\pm .015}$ & $.675_{\pm .006}$ & $46.2\%$ & $.193_{\pm .005}$ & $.367_{\pm .024}$ & $.645_{\pm .015}$ & $40.2\%$ \\
CoDi     & $.293_{\pm .034}$ & $.189_{\pm .054}$ & $.855_{\pm .025}$ & $44.6\%$ & $.247_{\pm .042}$ & $.172_{\pm .063}$ & $.857_{\pm .031}$ & $42.5\%$ \\
Goggle   & $.833_{\pm .030}$ & $.533_{\pm .016}$ & $.827_{\pm .070}$ & $73.1\%$ & $.882_{\pm .023}$ & $.542_{\pm .024}$ & $.807_{\pm .053}$ & $74.4\%$ \\
GReaT    & $.249_{\pm .015}$ & $.572_{\pm .289}$ & $.624_{\pm .066}$ & $48.2\%$ & $.155_{\pm .028}$ & $.380_{\pm .126}$ & $.543_{\pm .075}$ & $35.9\%$ \\
SMOTE    & $.321_{\pm .006}$ & $.405_{\pm .028}$ & $.648_{\pm .021}$ & $45.8\%$ & $.254_{\pm .011}$ & $.381_{\pm .026}$ & $.589_{\pm .009}$ & $40.8\%$ \\
STaSy    & $.261_{\pm .045}$ & $.468_{\pm .123}$ & $.436_{\pm .092}$ & $38.8\%$ & $.182_{\pm .069}$ & $.451_{\pm .113}$ & $.433_{\pm .140}$ & $35.5\%$ \\
TabDDPM  & $.261_{\pm .006}$ & $.337_{\pm .020}$ & $.558_{\pm .041}$ & $38.5